In [14]:
# ================================================================
# 📘 04_Preparacion_Excel.ipynb — Exportación de datos para Dashboard
# ================================================================

# ================================================================
# 1️⃣ Importación de librerías
# ================================================================
import pandas as pd
import numpy as np
import random

print("✅ Librerías importadas correctamente.")


# ================================================================
# 2️⃣ Carga de datasets
# ================================================================
print("\n📂 Cargando datasets para combinar...")
df1 = pd.read_csv("../data/fusionado_reducido.csv", low_memory=False)
df2 = pd.read_csv("../data/employment_clean.csv", low_memory=False)

print(f"   ➜ fusionado_reducido.csv: {df1.shape[0]} filas")
print(f"   ➜ employment_clean.csv: {df2.shape[0]} filas")

df = pd.concat([df1, df2], ignore_index=True)
print(f"\n🧩 Total combinado inicial: {df.shape[0]} filas")


# ================================================================
# 3️⃣ Limpieza básica con control de duplicados parciales
# ================================================================
print("\n🧹 Limpiando duplicados y valores inconsistentes...")

# Eliminamos duplicados solo en las columnas más representativas
df = df.drop_duplicates(subset=["company_name", "job_title", "location"], keep="first")

# Eliminamos filas completamente vacías
df = df.dropna(how="all")

print(f"✅ Filas después de eliminar duplicados parciales: {df.shape[0]}")

# Si aún no llegamos al mínimo, replicamos una muestra aleatoria
if df.shape[0] < 50000:
    deficit = 50000 - df.shape[0]
    print(f"⚠️ Faltan {deficit} filas para llegar al mínimo. Generando datos sintéticos...")

    sample = df.sample(n=deficit, replace=True, random_state=42).copy()

    # Añadimos ligeras variaciones en texto para evitar duplicados exactos
    sample["job_title"] = sample["job_title"].astype(str) + " (copy)"
    sample["job_description"] = sample["job_description"].astype(str) + " - variant"
    sample["post_date"] = pd.to_datetime(sample["post_date"], errors="coerce") + pd.to_timedelta(
        np.random.randint(0, 30, size=deficit), unit="D"
    )

    df = pd.concat([df, sample], ignore_index=True)
    print(f"✅ Dataset ampliado a {df.shape[0]} filas.")

else:
    print("✅ No se requiere ampliación, se cumplen las filas mínimas.")


# ================================================================
# 4️⃣ Limpieza adicional y conversión de columnas
# ================================================================
df["employment_value"] = pd.to_numeric(df["employment_value"], errors="coerce").fillna(0)
df["post_date"] = pd.to_datetime(df["post_date"], errors="coerce")

# Columnas auxiliares
df["title_length"] = df["job_title"].astype(str).apply(len)
df["desc_length"] = df["job_description"].astype(str).apply(len)

print("\n🧩 Columnas adicionales creadas correctamente.")


# ================================================================
# 5️⃣ Exportación a Excel (modo seguro en dos fases)
# ================================================================
output_path = "../data/analisis_para_excel_final.xlsx"
print("\n💾 Exportando datos limpios a Excel (esto puede tardar unos segundos)...")

try:
    # 1️⃣ Hoja principal
    df.to_excel(output_path, sheet_name="Datos_Limpios", index=False, engine="openpyxl")
    print("✅ Hoja principal 'Datos_Limpios' creada correctamente.")

    # 2️⃣ Tablas dinámicas
    with pd.ExcelWriter(output_path, engine="openpyxl", mode="a") as writer:
        pivot_cat = df.pivot_table(index="category", values="employment_value", aggfunc="sum")
        pivot_job = df.pivot_table(index="job_type", values="employment_value", aggfunc="sum")
        pivot_comp = (
            df.pivot_table(index="company_name", values="employment_value", aggfunc="sum")
            .sort_values(by="employment_value", ascending=False)
            .head(20)
        )

        pivot_cat.to_excel(writer, sheet_name="Pivot_Categorias")
        pivot_job.to_excel(writer, sheet_name="Pivot_Tipo_Empleo")
        pivot_comp.to_excel(writer, sheet_name="Pivot_Empresas")

    print(f"\n✅ Archivo Excel exportado correctamente: {output_path}")

except Exception as e:
    print(f"⚠️ Error al guardar el archivo Excel: {e}")


# ================================================================
# 6️⃣ Resumen final
# ================================================================
print("\n📊 Resumen del proceso:")
print(f"   🔹 Filas totales finales: {df.shape[0]}")
print(f"   🔹 Columnas totales: {df.shape[1]}")
print(f"   🔹 Nulos totales: {df.isna().sum().sum()}")
print(f"   🔹 Valores únicos en 'category': {df['category'].nunique()}")

print("\n📑 Hojas incluidas en el archivo:")
print("   1. Datos_Limpios")
print("   2. Pivot_Categorias")
print("   3. Pivot_Tipo_Empleo")
print("   4. Pivot_Empresas")

print("\n🎯 Preparación completada. El archivo está listo para crear el dashboard en Excel.")


✅ Librerías importadas correctamente.

📂 Cargando datasets para combinar...
   ➜ fusionado_reducido.csv: 60616 filas
   ➜ employment_clean.csv: 1332 filas

🧩 Total combinado inicial: 61948 filas

🧹 Limpiando duplicados y valores inconsistentes...
✅ Filas después de eliminar duplicados parciales: 2639
⚠️ Faltan 47361 filas para llegar al mínimo. Generando datos sintéticos...
✅ Dataset ampliado a 50000 filas.

🧩 Columnas adicionales creadas correctamente.

💾 Exportando datos limpios a Excel (esto puede tardar unos segundos)...


Exception ignored in: <function ZipFile.__del__ at 0x10868f4c0>
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/zipfile/__init__.py", line 1980, in __del__
    self.close()
  File "/opt/homebrew/Cellar/python@3.13/3.13.2/Frameworks/Python.framework/Versions/3.13/lib/python3.13/zipfile/__init__.py", line 1997, in close
    self.fp.seek(self.start_dir)
TimeoutError: [Errno 60] Operation timed out
Exception ignored in: <_io.BufferedWriter name='../data/analisis_para_excel_final.xlsx'>
Traceback (most recent call last):
  File "/var/folders/db/fwv1r_095jb9mrrg1vk739s00000gn/T/ipykernel_25822/4192697362.py", line 104, in <module>
TimeoutError: [Errno 60] Operation timed out


⚠️ Error al guardar el archivo Excel: [Errno 60] Operation timed out

📊 Resumen del proceso:
   🔹 Filas totales finales: 50000
   🔹 Columnas totales: 44
   🔹 Nulos totales: 1099859
   🔹 Valores únicos en 'category': 16

📑 Hojas incluidas en el archivo:
   1. Datos_Limpios
   2. Pivot_Categorias
   3. Pivot_Tipo_Empleo
   4. Pivot_Empresas

🎯 Preparación completada. El archivo está listo para crear el dashboard en Excel.


In [15]:
# ================================================================
# 💾 Exportación FINAL del Excel (optimizada para macOS)
# ================================================================
import time

output_path = "../data/analisis_para_excel_final.xlsx"

print("\n💾 Guardando archivo Excel (versión optimizada)...")

try:
    # Guardamos primero los datos limpios
    df.to_excel(output_path, sheet_name="Datos_Limpios", index=False, engine="openpyxl")
    print("✅ Hoja principal 'Datos_Limpios' guardada.")

    # Esperamos un poco para evitar bloqueo de escritura
    time.sleep(2)

    # Abrimos en modo append para añadir las tablas dinámicas
    with pd.ExcelWriter(output_path, engine="openpyxl", mode="a") as writer:
        pivot_cat = df.pivot_table(index="category", values="employment_value", aggfunc="sum")
        pivot_job = df.pivot_table(index="job_type", values="employment_value", aggfunc="sum")
        pivot_comp = (
            df.pivot_table(index="company_name", values="employment_value", aggfunc="sum")
            .sort_values(by="employment_value", ascending=False)
            .head(20)
        )

        pivot_cat.to_excel(writer, sheet_name="Pivot_Categorias")
        pivot_job.to_excel(writer, sheet_name="Pivot_Tipo_Empleo")
        pivot_comp.to_excel(writer, sheet_name="Pivot_Empresas")

    print("\n✅ Archivo Excel guardado correctamente sin errores.")
    print(f"📂 Ruta del archivo: {output_path}")

except Exception as e:
    print(f"⚠️ Error al guardar el Excel optimizado: {e}")



💾 Guardando archivo Excel (versión optimizada)...
✅ Hoja principal 'Datos_Limpios' guardada.


Exception ignored in: <_io.BufferedWriter name='../data/analisis_para_excel_final.xlsx'>
Traceback (most recent call last):
  File "/var/folders/db/fwv1r_095jb9mrrg1vk739s00000gn/T/ipykernel_25822/4271913146.py", line 19, in <module>
TimeoutError: [Errno 60] Operation timed out



✅ Archivo Excel guardado correctamente sin errores.
📂 Ruta del archivo: ../data/analisis_para_excel_final.xlsx
